Import

In [103]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup


In [104]:
soup = ""
with open("data.xml", "r", encoding="utf-8") as file:
    content = file.read()
    soup = BeautifulSoup(content, "xml")
    file.close()


#ProductTree

In [105]:
library = soup.find('ProductTree')
productdb = {'ProductID':[], 'ProductName':[], 'Type':[], 'Name':[]}


def process_node(node, productdb):

    if node.name is not None:
        if 'ProductID' in node.attrs:
            productdb['ProductID'].append(node.attrs['ProductID'])
            productdb['ProductName'].append(node.text)
        
        if 'Type' not in node.attrs and node.name != 'ProductTree':
            if 'Type' in node.parent.attrs:
                 productdb['Type'].append(node.parent.attrs['Type'])
                 productdb['Name'].append(node.parent.attrs['Name'])
            else:
                productdb['Type'].append(None)
                productdb['Name'].append(None)

        for child in node.children:
            process_node(child, productdb)


process_node(library, productdb)
productdb = pd.DataFrame(productdb)
productdb.to_excel('data.xlsx', index=False)
print(productdb)


    ProductID                                        ProductName  \
0       11568         Windows 10 Version 1809 for 32-bit Systems   
1       11569      Windows 10 Version 1809 for x64-based Systems   
2       11570    Windows 10 Version 1809 for ARM64-based Systems   
3       11571                                Windows Server 2019   
4       11572     Windows Server 2019 (Server Core installation)   
..        ...                                                ...   
177     12139                                CBL Mariner 2.0 x64   
178     12140                                CBL Mariner 2.0 ARM   
179      9312  Windows Server 2008 for 32-bit Systems Service...   
180      9318  Windows Server 2008 for x64-based Systems Serv...   
181      9344  Windows Server 2008 for x64-based Systems Serv...   

               Type     Name  
0    Product Family  Windows  
1    Product Family  Windows  
2    Product Family  Windows  
3    Product Family  Windows  
4    Product Family  Windows

#Vulnerability

In [106]:

vulnerability_db = {"Ordinal":[], "CVE":[]}
vulnerability_db_status = {"VulnerabilityFK":[], "StatusType": [], "ProductID":[]}
vulnerability_db_notes = {"VulnerabilityFK":[], "Title":[], "Type":[], "Ordinal":[], "Note":[]}
vulnerability_db_threats = {"VulnerabilityFK":[], "Type":[], "Description":[], "ProductID":[]}
def vulnerability(node, vulnerability_db):
    if node.name is not None:

        def get_x_parent(res, x):
            for i in range(x):
                res = res.parent
            return res

        if 'Ordinal' in node.attrs and node.name == 'Vulnerability':
            vulnerability_db['Ordinal'].append(node.attrs['Ordinal'])
        
        if node.name == 'CVE':
            vulnerability_db['CVE'].append(node.text)

        if node.name == 'ProductID' and node.parent.name == "Status":
            vulnerability_db_status['ProductID'].append(node.text)
            vulnerability_db_status['StatusType'].append(node.parent.attrs['Type'])
            vulnerability_db_status['VulnerabilityFK'].append(len(vulnerability_db['Ordinal'])-1)

        if node.name == 'Note' and node.parent.name == "Notes":
            vulnerability_db_notes['VulnerabilityFK'].append(len(vulnerability_db['Ordinal'])-1)
            vulnerability_db_notes["Title"].append(node.attrs['Title'])
            vulnerability_db_notes["Type"].append(node.attrs['Type'])
            vulnerability_db_notes["Ordinal"].append(node.attrs['Ordinal'])
            vulnerability_db_notes["Note"].append(node.text)

        if node.name == 'Threat' and node.parent.name == "Threats":
            vulnerability_db_threats['VulnerabilityFK'].append(len(vulnerability_db['Ordinal'])-1)
            vulnerability_db_threats['Type'].append(node.attrs['Type'])

            if node.find('ProductID'):
                
                for i in node.children:
                    if i.name == 'ProductID': 
                        vulnerability_db_threats['ProductID'].append(i.text)   

                    if i.name == 'Description' and i.text != '': 
                        vulnerability_db_threats['Description'].append(i.text)
                    
                    if i.name == 'Description' and i.text == '': 
                        vulnerability_db_threats['Description'].append(None)
            else:
                vulnerability_db_threats['ProductID'].append(None) 
                vulnerability_db_threats['Description'].append(node.find("Description").text)



        for child in node.children:
            vulnerability(child, vulnerability_db)

lib = soup.find('cvrfdoc')
lib = lib.children
for child in lib:
    if child.name == "Vulnerability":
        vulnerability(child, vulnerability_db)
vulnerability_db = pd.DataFrame(vulnerability_db)
vulnerability_db_status = pd.DataFrame(vulnerability_db_status)
vulnerability_db_notes = pd.DataFrame(vulnerability_db_notes)
vulnerability_db_threats = pd.DataFrame(vulnerability_db_threats)
print(vulnerability_db_threats)
# print(len(vulnerability_db_threats["VulnerabilityFK"]), len(vulnerability_db_threats["Description"]), len(vulnerability_db_threats["ProductID"]))


      VulnerabilityFK            Type Description ProductID
0                   0          Impact        None     12139
1                   0          Impact        None     12140
2                   0        Severity        None     12139
3                   0        Severity        None     12140
4                   0  Exploit Status     DOS:N/A      None
...               ...             ...         ...       ...
5250              227        Severity        None     11655
5251              227  Exploit Status     DOS:N/A      None
5252              228          Impact        None     11655
5253              228        Severity        None     11655
5254              228  Exploit Status     DOS:N/A      None

[5255 rows x 4 columns]
